In [13]:
import pandas as pd
views_text=pd.read_csv('景点(无特殊字符).csv').sample(n=10000)
views_text=views_text.dropna().reset_index().drop('index',axis=1)
import numpy as np

In [14]:
# 去除特殊字符
import string
import numpy as np
import re

views_new = []
for view in views_text['txt']:  # views_text[0]的第0列，指
    view = view.replace(' ', '') #将空格去掉
    view = 'bbb' + view
    view = re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",view)
    views_new.append(list(view))

print(len(views_new))

9975


In [15]:
# 生成X和Y的矩阵
XY  =[]
for view in views_new:
    for i in range(len(view) - 3):
        x1 = view[i]
        x2 = view[i+1]
        x3 = view[i+2]
        
        y = view[i+3]
        XY.append([x1, x2, x3, y])

# 展示整理后的X和Y的形式

print("原始：")
print(views_text['txt'][480])
print("\n")
print("训练数据：")
print(["X1", "X2", "X3", "Y"])
for i in range(3800, 3810):
    print(XY[i])

原始：
没按照景区景点的路线讲解不值


训练数据：
['X1', 'X2', 'X3', 'Y']
['点', '还', '会', '转']
['还', '会', '转', '身']
['会', '转', '身', '喵']
['转', '身', '喵', '你']
['身', '喵', '你', '叫']
['喵', '你', '叫', '你']
['你', '叫', '你', '走']
['叫', '你', '走', '快']
['你', '走', '快', '点']
['走', '快', '点', '跟']


In [16]:
# 文字编码
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(views_new)
print(tokenizer.word_index)

#tokenizer默认把0这个索引留给停止词了，其它的词是从1开始索引的。然后分类的损失又默认0是第一类
#所以导致假设我们有三个字：a，b，c。他们的索引分别是1，2，3，没有0。这对keras要使用一个4分类才行
vocab_size = len(tokenizer.word_index) + 1

print(vocab_size)

{'b': 1, '的': 2, '不': 3, '一': 4, '是': 5, '很': 6, '了': 7, '有': 8, '好': 9, '去': 10, '人': 11, '看': 12, '景': 13, '个': 14, '还': 15, '大': 16, '多': 17, '上': 18, '就': 19, '票': 20, '点': 21, '到': 22, '游': 23, '得': 24, '以': 25, '可': 26, '错': 27, '在': 28, '山': 29, '来': 30, '西': 31, '方': 32, '没': 33, '下': 34, '时': 35, '要': 36, '也': 37, '地': 38, '值': 39, '车': 40, '安': 41, '这': 42, '小': 43, '解': 44, '都': 45, '里': 46, '太': 47, '门': 48, '感': 49, '我': 50, '常': 51, '非': 52, '天': 53, '城': 54, '物': 55, '区': 56, '0': 57, '讲': 58, '便': 59, '历': 60, '史': 61, '观': 62, '中': 63, '馆': 64, '和': 65, '面': 66, '建': 67, '行': 68, '美': 69, '过': 70, '后': 71, '觉': 72, '比': 73, '次': 74, '古': 75, '能': 76, '子': 77, '真': 78, '玩': 79, '走': 80, '为': 81, '1': 82, '导': 83, '华': 84, '最': 85, '文': 86, '心': 87, '但': 88, '出': 89, '说': 90, '马': 91, '然': 92, '道': 93, '自': 94, '园': 95, '开': 96, '间': 97, '体': 98, '会': 99, '风': 100, '家': 101, '路': 102, '墙': 103, '们': 104, '么': 105, '之': 106, '色': 107, '进': 108, '买': 109, '程': 110, '挺': 11

In [17]:
XY_digit = np.array(tokenizer.texts_to_sequences(XY))
X_digit = XY_digit[:, :3]
Y_digit = XY_digit[:, 3]

for i in range(1327, 1337):
    print("{:<35}".format(str(XY[i])), "\t", "{:<30}".format(str(list(X_digit[i]))),"\t", Y_digit[i])

['一', '把', '睡', '在']                	 [4, 531, 1141]                 	 28
['把', '睡', '在', '老']                	 [531, 1141, 28]                	 177
['睡', '在', '老', '炕']                	 [1141, 28, 177]                	 1979
['在', '老', '炕', '头']                	 [28, 177, 1979]                	 464
['老', '炕', '头', '的']                	 [177, 1979, 464]               	 2
['炕', '头', '的', '意']                	 [1979, 464, 2]                 	 121
['头', '的', '意', '境']                	 [464, 2, 121]                  	 299
['的', '意', '境', '边']                	 [2, 121, 299]                  	 214
['意', '境', '边', '喝']                	 [121, 299, 214]                	 1016
['境', '边', '喝', '茶']                	 [299, 214, 1016]               	 1005


In [6]:
# Embedding + 线性模型
from tensorflow.keras.layers import Input, Embedding
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, Activation, Embedding, Flatten
hidden_size = 300

inp = Input(shape=(3,))
x = Embedding(vocab_size, hidden_size)(inp)
x = Flatten()(x)
x = Dense(vocab_size)(x)
pred = Activation('softmax')(x)

lstm_model = Model(inp, pred)
lstm_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 embedding (Embedding)       (None, 3, 300)            779400    
                                                                 
 flatten (Flatten)           (None, 900)               0         
                                                                 
 dense (Dense)               (None, 2598)              2340798   
                                                                 
 activation (Activation)     (None, 2598)              0         
                                                                 
Total params: 3,120,198
Trainable params: 3,120,198
Non-trainable params: 0
_________________________________________________________________


In [15]:
# 训练模型
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_digit,Y_digit,test_size=0.2, random_state=0)

from tensorflow.keras.optimizers import Adam
lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001))
lstm_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=100, epochs=10)
lstm_model.save('CNNview' + 'Model')
# model.save_weights('CNN.h5') # 文件类型是HDF5

Epoch 1/10
585/585 [==============================] - 26s 43ms/step - loss: 6.3246 - val_loss: 5.5651
Epoch 2/10
585/585 [==============================] - 28s 48ms/step - loss: 4.9229 - val_loss: 4.9871
Epoch 3/10
585/585 [==============================] - 28s 47ms/step - loss: 4.0535 - val_loss: 4.7750
Epoch 4/10
585/585 [==============================] - 25s 43ms/step - loss: 3.3760 - val_loss: 4.7341
Epoch 5/10
585/585 [==============================] - 26s 44ms/step - loss: 2.8388 - val_loss: 4.7752
Epoch 6/10
585/585 [==============================] - 26s 45ms/step - loss: 2.4262 - val_loss: 4.8702
Epoch 7/10
585/585 [==============================] - 27s 46ms/step - loss: 2.1129 - val_loss: 4.9920
Epoch 8/10
585/585 [==============================] - 27s 46ms/step - loss: 1.8758 - val_loss: 5.1283
Epoch 9/10
585/585 [==============================] - 26s 45ms/step - loss: 1.6939 - val_loss: 5.2809
Epoch 10/10
585/585 [==============================] - 28s 48ms/step - loss: 1.555

INFO:tensorflow:Assets written to: CNNviewModel\assets


INFO:tensorflow:Assets written to: CNNviewModel\assets


In [36]:
# 模型效果检验
from tensorflow.keras.models import load_model
model = load_model('CNNview' + 'model')
# model.load_weights('CNN.h5')
sample_text = ['雨', '水', '落'] 
print(sample_text)
sample_index = tokenizer.texts_to_sequences(sample_text)
print(sample_index)
word_prob = model.predict(np.array(sample_index).reshape(1, 3))[0]
print(tokenizer.index_word[word_prob.argmax()], word_prob.max())

['雨', '水', '落']
[[76], [159], [198]]
1/1 [==============================] - 0s 30ms/step
下 0.14548357


In [26]:
# 模型应用
view_incomplete = 'bbb导游*****************************'
view_index = []
view_text = ''
for i in range(len(view_incomplete)):
    current_word = view_incomplete[i]
    
    if  current_word != '*':
        # 给定的词
        index = tokenizer.word_index[current_word]
    else:
        # 根据前三个词预测 *
        x = view_index[-3:]
        y = lstm_model.predict(np.expand_dims(x, axis=0))[0]
        index = y.argmax()
        current_word = tokenizer.index_word[index]


    view_index.append(index)
    view_text = view_text + current_word
        

view_text = view_text[3:]
print(view_text)

1/1 [==============================] - 0s 53ms/step
导游讲解很有意义的人文景观充分为西安的地标性建筑工艺别具一格是


In [56]:
list('乐乐服务周到非常贴心兵马俑震撼人心')

['乐',
 '乐',
 '服',
 '务',
 '周',
 '到',
 '非',
 '常',
 '贴',
 '心',
 '兵',
 '马',
 '俑',
 '震',
 '撼',
 '人',
 '心']

In [18]:
# 数据的输入与展示
import string
import numpy as np

views=[]
for view in views_text.txt:
    #print(view)
    view = view.replace(' ', '') #将空格去掉
    view = view.replace('\n', '') #将换行符去掉
    #print(list(view))
    views.append(list(view))
    

In [19]:
# 数据整理：文字编码
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(views)
views_digit = tokenizer.texts_to_sequences(views)
vocab_size = len(tokenizer.word_index) + 1 #加上停止词0
vocab_size #有多少个不同的字

3431

In [20]:
#补全数据：为了将所有的诗放在一个M*N的np.array中，将每一首诗补0到同样的长度
views_digit = pad_sequences(views_digit, maxlen=50, padding='post')
print("原始评论")
print(views[864])
print("\n")
print("编码+补全后的结果")
print(views_digit[864])

原始评论
['门', '票', '太', '贵', '了', '携', '程', '优', '惠', '价', '也', '6', '0', '元', '非', '大', '众', '消', '费', '水', '平', '同', '等', '次', '的', '广', '州', '白', '云', '山', '风', '景', '区', '5', '元', '市', '内', '公', '园', '免', '费', '广', '州', '住', '宿', '也', '不', '贵']


编码+补全后的结果
[  47   19   46  179    6  245  109  373  489  124   36  230   56  178
   51   15  743  823  130  196  438  398  223   73    1  542 1340  334
  571   28   99   12   55  161  178  269  153  128   94  248  130  542
 1340  476  914   36    2  179    0    0]


In [21]:
# 对齐X和Y
X = views_digit[:, :-1]
Y = views_digit[:, 1:]

print(views_digit.shape)
print(X.shape)
print(Y.shape)

print("X示例", "\t", "Y示例")

for i in range(10):
    print(X[0][i], "\t", Y[0][i])
    
print("...", "\t", "...")

(9975, 50)
(9975, 49)
(9975, 49)
X示例 	 Y示例
14 	 8
8 	 201
201 	 380
380 	 895
895 	 1
1 	 5
5 	 579
579 	 673
673 	 172
172 	 330
... 	 ...


In [24]:
# 把$Y$变成One-Hot向量
print(vocab_size)
from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y, num_classes=vocab_size)
print(Y.shape)

3431
(9975, 49, 3431)


In [25]:
# 确定空间维度
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Embedding, Activation, BatchNormalization
embedding_size = 100
hidden_size = 200

# 构建RNN模型
inp = Input(shape=(49,))

# Encoder
x = Embedding(vocab_size, embedding_size, mask_zero=True)(inp)
x = SimpleRNN(hidden_size,return_sequences=True)(x)

# prediction
x = Dense(vocab_size)(x)
pred = Activation('softmax')(x)

model = Model(inp, pred)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 49)]              0         
                                                                 
 embedding (Embedding)       (None, 49, 100)           343100    
                                                                 
 simple_rnn (SimpleRNN)      (None, 49, 200)           60200     
                                                                 
 dense (Dense)               (None, 49, 3431)          689631    
                                                                 
 activation (Activation)     (None, 49, 3431)          0         
                                                                 
Total params: 1,092,931
Trainable params: 1,092,931
Non-trainable params: 0
_________________________________________________________________


In [77]:
# 模型训练
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.fit(X, Y, epochs=10, batch_size=30, validation_split=0.2)
model.save('RNNview' + 'Model')

Epoch 1/10
42/42 [==============================] - 7s 107ms/step - loss: 5.6616 - accuracy: 0.0967 - val_loss: 6.2067 - val_accuracy: 0.0857
Epoch 2/10
42/42 [==============================] - 5s 112ms/step - loss: 5.4884 - accuracy: 0.1150 - val_loss: 6.1380 - val_accuracy: 0.1010
Epoch 3/10
42/42 [==============================] - 6s 132ms/step - loss: 5.3452 - accuracy: 0.1319 - val_loss: 6.0669 - val_accuracy: 0.1106
Epoch 4/10
42/42 [==============================] - 5s 131ms/step - loss: 5.2026 - accuracy: 0.1470 - val_loss: 6.0052 - val_accuracy: 0.1203
Epoch 5/10
42/42 [==============================] - 5s 120ms/step - loss: 5.0609 - accuracy: 0.1607 - val_loss: 5.9613 - val_accuracy: 0.1268
Epoch 6/10
42/42 [==============================] - 5s 126ms/step - loss: 4.9205 - accuracy: 0.1747 - val_loss: 5.9232 - val_accuracy: 0.1333
Epoch 7/10
42/42 [==============================] - 5s 128ms/step - loss: 4.7926 - accuracy: 0.1865 - val_loss: 5.8847 - val_accuracy: 0.1381
Epoch 

INFO:tensorflow:Assets written to: RNNviewModel\assets


INFO:tensorflow:Assets written to: RNNviewModel\assets


In [78]:
# 模型应用
from tensorflow.keras.models import load_model
model = load_model('RNNview' + 'model')

poem_incomplete = '体验***************'
poem_index = []
poem_text = ''
for i in range(len(poem_incomplete)):
    current_word = poem_incomplete[i]
    
    if  current_word != '*':
        index = tokenizer.word_index[current_word]
        
    else:
        x = np.expand_dims(poem_index, axis=0)
        x = pad_sequences(x, maxlen=49, padding='post')
        y = model.predict(x)[0, i]
        
        y[0] = 0            #去掉停止词
        index = y.argmax()
        current_word = tokenizer.index_word[index]
        
    poem_index.append(index)
    poem_text = poem_text + current_word
        
poem_text = poem_text[0:]
print(poem_text)

1/1 [==============================] - 0s 24ms/step
体验景点不错的地方便快捷很好便宜人


In [21]:
# 构建LSTM的模型
# from keras.models import Model
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Activation, BatchNormalization
from tensorflow.keras import Model

hidden_size1 = 300
hidden_size2 = 100

inp = Input(shape=(49,))

# Encoder
x = Embedding(vocab_size, hidden_size1, input_length=49, mask_zero=True)(inp)
x = LSTM(hidden_size2, return_sequences=True)(x)

# prediction
x = Dense(vocab_size)(x)
pred = Activation('softmax')(x)

model = Model(inp, pred)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 49)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 49, 300)           780900    
                                                                 
 lstm_1 (LSTM)               (None, 49, 100)           160400    
                                                                 
 dense_3 (Dense)             (None, 49, 2603)          262903    
                                                                 
 activation_3 (Activation)   (None, 49, 2603)          0         
                                                                 
Total params: 1,204,203
Trainable params: 1,204,203
Non-trainable params: 0
_________________________________________________________________


In [30]:
# 模型训练
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
model.fit(X, Y, epochs=100, batch_size=30, validation_split=0.2)
model.save('LSTMview' + 'Model')

Epoch 1/100
42/42 [==============================] - 28s 507ms/step - loss: 2.9858 - accuracy: 0.4102 - val_loss: 6.1997 - val_accuracy: 0.1797
Epoch 2/100
42/42 [==============================] - 18s 437ms/step - loss: 2.9351 - accuracy: 0.4173 - val_loss: 6.2085 - val_accuracy: 0.1784
Epoch 3/100
42/42 [==============================] - 18s 419ms/step - loss: 2.9092 - accuracy: 0.4221 - val_loss: 6.2203 - val_accuracy: 0.1796
Epoch 4/100
42/42 [==============================] - 15s 359ms/step - loss: 2.8887 - accuracy: 0.4259 - val_loss: 6.2342 - val_accuracy: 0.1788
Epoch 5/100
42/42 [==============================] - 14s 340ms/step - loss: 2.8642 - accuracy: 0.4293 - val_loss: 6.2479 - val_accuracy: 0.1797
Epoch 6/100
42/42 [==============================] - 16s 370ms/step - loss: 2.8443 - accuracy: 0.4326 - val_loss: 6.2663 - val_accuracy: 0.1782
Epoch 7/100
42/42 [==============================] - 19s 440ms/step - loss: 2.8290 - accuracy: 0.4344 - val_loss: 6.2781 - val_accuracy:

INFO:tensorflow:Assets written to: LSTMviewModel\assets


INFO:tensorflow:Assets written to: LSTMviewModel\assets


In [29]:
# 应用模型
from tensorflow.keras.models import load_model

model = load_model('LSTMview' + 'model')

view_incomplete = '西安*************'
view_index = []
view_text = ''
for i in range(len(view_incomplete)):
    current_word = view_incomplete[i]

    if current_word != '*':
        index = tokenizer.word_index[current_word]

    else:
        x = np.expand_dims(view_index, axis=0)
        x = pad_sequences(x, maxlen=49, padding='post')
        y = model.predict(x)[0, i]

        y[0] = 0  # 去掉停止词
        index = y.argmax()
        current_word = tokenizer.index_word[index]

    view_index.append(index)
    view_text = view_text + current_word

view_text = view_text[0:]
print(view_text)

1/1 [==============================] - 0s 34ms/step
西安市市区内容区有一个地方处风
